In [1]:
from openai import AzureOpenAI
import time
import bot as bot
from eval_utils import *
import re

import warnings
warnings.filterwarnings("ignore", message="CropBox missing from /Page") 
warnings.filterwarnings("ignore", message=".*ScriptRunContext.*")

/opt/anaconda3/envs/bcwds4/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2025-05-26 13:46:43.545175: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ
https://ai-bcds.openai.azure.com/


2025-05-26 13:46:49.754 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 13:46:51.165 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 13:46:51.166 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 13:46:51.200 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-26 13:46:51.427 
  command:

    streamlit run /opt/anaconda3/envs/bcwds4/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-26 13:46:51.428 Thread 'MainThread': missing ScriptRunContext! This warning can be 

**Extract Q&A from file**

In [2]:
qa_pairs = extract_qa_pairs_from_pdf('./docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


# FidChat

In [3]:
instructions = bot.instructions

In [4]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint="https://ai-bcds.openai.azure.com/",
    api_key="8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
    api_version="2024-05-01-preview"
)

assistant = client.beta.assistants.create(
    model="gpt-4o-mini", 
    instructions = instructions,
    tool_resources={"code_interpreter":{"file_ids":["assistant-WZQHoFwobKHT1ZWKkEAGqm","assistant-MsQgxjsadXVcZKLw2fJwNh","assistant-LpaoUAR5Rq3MgXfMSU4i9A","assistant-KW358eNBGny1N5DM8LoBaC","assistant-CiKqiv2zmbtF8FH2iQAKAw","assistant-9yhV7cKx5ynoLqoPbs1B67","assistant-5QanyuMMkjkkfVUv6rkCfM"]}},
    temperature=0.2,
    top_p=1
    )

In [5]:
def fidchat(client, assistant_id, message, instructions):
    # Create a new thread
    thread = client.beta.threads.create()

    # Add message (with optional system instructions)
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=instructions + "\n\n" + message
    )

    # Run assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait until response is ready
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Return the assistant's response
    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                raw_response = msg.content[0].text.value
                clean_response = re.sub(r"【\d+:\d+†source】", "", raw_response).strip()
                return clean_response
    return "[ERROR] Failed to get assistant response."


# EvalChat

In [20]:
client2 = AzureOpenAI(
  azure_endpoint = "https://ai-bcds.openai.azure.com/",
  api_key= "8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ3w3AAABACOGLa23",
  api_version="2024-05-01-preview"
)

assistant2 = client.beta.assistants.create(
  model="gpt-4o-mini", 
  instructions="",
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":["vs_ezpssNVZDpwb0RjgcxSwbNZz"]}},
  temperature=1,
  top_p=1
)

In [37]:
import time
import re

def eval_chat(client, assistant_id, question, expected_answer, actual_answer):
    instructions = instructions2

    # Create a new thread
    thread = client.beta.threads.create()

    # Prepare the prompt
    prompt = f"{instructions}\n\nQuestion: {question}\nExpected Answer: {expected_answer}\nActual Answer: {actual_answer}"

    # Send the message
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Wait until it's done
    while run.status in ["queued", "in_progress", "cancelling"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve and return the response
    if run.status == "completed":
        messages_list = client.beta.threads.messages.list(thread_id=thread.id)
        for msg in reversed(messages_list.data):
            if msg.role == "assistant":
                response = msg.content[0].text.value.strip()
                return response
    return "[ERROR] Failed to get assistant response."


In [29]:
# qa_pairs
#instructions2 = 'Give a similarity score between expected answer and actual answer. \
#                 Output should be a number between 0-10. Do not write anything else'

#actual_answer = 'Hello'
#question = qa_pairs[0][0]

In [22]:
#eval_chat(client2, assistant2.id, question, actual_answer, instructions2)

'0'

# Evaluation

In [36]:
# qa_pairs
instructions2 = 'Give a general similarity score between expected answer and actual answer. \
                 The answers are supposed to be similar and transmite more or less the same message. \
                 If the actual answer looks more complete than the expected answer, the similarity score should be higher.\
                 Output should be a whole number between 0-10. Do not write anything else'

In [38]:
results = []

for idx, (question, expected_answer) in enumerate(qa_pairs):
    print(f"Evaluating Q{idx+1}...")
    
    # Get assistant's answer
    actual_answer = fidchat(client, assistant.id, question, instructions)
    evaluation = eval_chat(client2, assistant2.id, question, expected_answer, actual_answer)

    results.append({
        "question": question,
        "expected_answer": expected_answer,
        "actual_answer": actual_answer,
        "evaluation": evaluation
    })
    


Evaluating Q1...
Evaluating Q2...
Evaluating Q3...
Evaluating Q4...
Evaluating Q5...
Evaluating Q6...
Evaluating Q7...
Evaluating Q8...
Evaluating Q9...
Evaluating Q10...
Evaluating Q11...
Evaluating Q12...
Evaluating Q13...
Evaluating Q14...
Evaluating Q15...
Evaluating Q16...
Evaluating Q17...


In [39]:
for i in range(len(results)):
    print(f'Q{i+1}', results[i]["evaluation"])


Q1 6
Q2 9
Q3 8
Q4 7
Q5 8
Q6 7
Q7 8
Q8 7
Q9 8
Q10 4
Q11 3
Q12 1
Q13 4
Q14 7
Q15 8
Q16 4
Q17 8


In [40]:
for r in results:
    print("\n---")
    print("Q:", r["question"])
    print('')
    print("EA:", r["expected_answer"])
    print('')
    print("AA:", r["actual_answer"])
    print('')
    print("Evaluation:", r["evaluation"])


---
Q: O Fidelidade Savings é mais indicado para que perfil de risco?

EA: o Fidelidade Savings é adequado para clientes conservadores, através da sua opção Seguro, com capital e rendimento garantido. Para clientes que gostam moderadamente do risco, através da opção Proteção com garantia de 90% do capital investido no vencimento e exposição ao Índice Multiativos. Para clientes propensos ao risco através da opção Dinâmico com 100% de exposição ao Índice Multiativos. Também está disponível a opção ESG, que investe no índice Multiativos ESG, com risco mas ESG, para clientes com preocupações Ambientais, Sociais e de Governança.

AA: O **Fidelidade My Savings** é um produto de poupança que se destina a investidores com um perfil de risco **baixo a moderado**. Este produto é ideal para aqueles que procuram uma forma segura de poupança, com um retorno estável e previsível, sem expor o capital a grandes flutuações de mercado.

Se tiveres mais perguntas sobre as características do produto ou c

In [ ]:
'''Q: Posso escolher a componente que pretendo(Ativo ou Proteção)?

EA: Não. O contrato apenas admite a recomposição automática, pelo que o Tomador do Seguro não tem direito a alterar a composição do investimento. Fiscalidade

AA: Sim, no produto **Fidelidade My Savings**, tens a possibilidade de escolher entre duas componentes: **Ativo** e **Proteção**. 

### Componentes:
- **Ativo**: Foca-se na valorização do capital, permitindo que o investimento cresça ao longo do tempo.
- **Proteção**: Oferece uma componente de segurança, garantindo uma proteção do capital investido.

Esta flexibilidade permite que adaptes o produto às tuas necessidades e objetivos financeiros. Se precisares de mais informações sobre cada componente ou sobre como funcionam, estou à disposição para ajudar!'''